In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


In [2]:
spark = SparkSession.builder.appName("spark_app").master("local[*]").getOrCreate()

21/11/14 08:48:36 WARN Utils: Your hostname, shivam-Vostro-3559 resolves to a loopback address: 127.0.1.1; using 192.168.0.105 instead (on interface wlx7cc2c61989aa)
21/11/14 08:48:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/14 08:48:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
if __name__ == "__main__":
    print("hello")

hello


In [4]:
df = spark.read.format('csv').option("header","true").option("inferSchema","true").option("path",r'data/merchants.csv').load()

In [5]:
df.printSchema()
df.show(truncate=False)

root
 |-- id: integer (nullable = true)
 |-- company_name: string (nullable = true)
 |-- telephone: string (nullable = true)
 |-- state: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- updated_at: string (nullable = true)

+------+---------------------------+------------+-----------+--------------------------+--------------------------+
|id    |company_name               |telephone   |state      |created_at                |updated_at                |
+------+---------------------------+------------+-----------+--------------------------+--------------------------+
|178307|MSI Kart                   |9319919988  |trial_ended|2021-01-02 09:33:02.178749|2021-01-02 09:39:08.302584|
|177863|Bellelyse Boutique         |000-000-0000|suspended  |2020-12-31 04:56:24.296919|2021-02-19 16:53:22.503263|
|144524|Rachid hamzaoui            |0639756718  |free       |2020-09-21 17:18:02.35187 |2020-10-22 11:15:07.122165|
|50742 |Thenaricalicollection      |5049394269  |suspende

In [6]:
df.selectExpr("*","cast(telephone as long) as phone_num")

DataFrame[id: int, company_name: string, telephone: string, state: string, created_at: string, updated_at: string, phone_num: bigint]

In [7]:
# cast column by adding column with different name
df.select(col('*'),col('telephone').cast('long').alias('phone_num'))

DataFrame[id: int, company_name: string, telephone: string, state: string, created_at: string, updated_at: string, phone_num: bigint]

In [8]:
## cast column by replacing existsting column
df.withColumn('telephone',col('telephone').cast('long'))

DataFrame[id: int, company_name: string, telephone: bigint, state: string, created_at: string, updated_at: string]

In [9]:
df.select(expr("cast( id  as string)"), "company_name")

DataFrame[id: string, company_name: string]

In [10]:
df.withColumn('created_at',col('created_at').cast('timestamp'))

DataFrame[id: int, company_name: string, telephone: string, state: string, created_at: timestamp, updated_at: string]

In [11]:
df = df.withColumn('telephone',col('telephone').cast('long')) \
    .withColumn('created_at',col('created_at').cast('timestamp')) \
    .withColumn('updated_at',col('updated_at').cast('timestamp'))

In [12]:
df.printSchema()
df.show(truncate=False)

root
 |-- id: integer (nullable = true)
 |-- company_name: string (nullable = true)
 |-- telephone: long (nullable = true)
 |-- state: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)

+------+---------------------------+----------+-----------+--------------------------+--------------------------+
|id    |company_name               |telephone |state      |created_at                |updated_at                |
+------+---------------------------+----------+-----------+--------------------------+--------------------------+
|178307|MSI Kart                   |9319919988|trial_ended|2021-01-02 09:33:02.178749|2021-01-02 09:39:08.302584|
|177863|Bellelyse Boutique         |null      |suspended  |2020-12-31 04:56:24.296919|2021-02-19 16:53:22.503263|
|144524|Rachid hamzaoui            |639756718 |free       |2020-09-21 17:18:02.35187 |2020-10-22 11:15:07.122165|
|50742 |Thenaricalicollection      |5049394269|suspended  |2019-0

In [13]:
df.select(to_date('updated_at').alias('updated_date'), \
         to_date('created_at').alias('created_date'), \
         (col('updated_at')-col('created_at')).alias('difference'), \
         datediff('updated_at','created_at').alias('date_difference'), \
         months_between('updated_at','created_at').alias('months_between'), \
         round(months_between('updated_at','created_at')).alias('rounded_months_bw'), \
         (months_between('updated_at','created_at') / lit(12)).alias('years_between')).show(truncate=False)

+------------+------------+---------------------------------------+---------------+--------------+-----------------+---------------------+
|updated_date|created_date|difference                             |date_difference|months_between|rounded_months_bw|years_between        |
+------------+------------+---------------------------------------+---------------+--------------+-----------------+---------------------+
|2021-01-02  |2021-01-02  |6 minutes 6.123835 seconds             |0              |0.0           |0.0              |0.0                  |
|2021-02-19  |2020-12-31  |1211 hours 56 minutes 58.206344 seconds|50             |1.62896431    |2.0              |0.13574702583333334  |
|2020-10-22  |2020-09-21  |737 hours 57 minutes 4.770295 seconds  |31             |1.02412821    |1.0              |0.0853440175         |
|2019-05-15  |2019-03-22  |1292 hours 57 minutes 53.19508 seconds |54             |1.77011387    |2.0              |0.14750948916666667  |
|2020-12-24  |2020-12-23  |

In [14]:
l = [(1,'abc',23),(2,'xyz',45)]
s = ['id','name','no']
spark.createDataFrame(l,schema=s).printSchema()
spark.createDataFrame(l,schema=s).show()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- no: long (nullable = true)



+---+----+---+
| id|name| no|
+---+----+---+
|  1| abc| 23|
|  2| xyz| 45|
+---+----+---+



In [15]:
sc = "id int, name string, no long"
spark.createDataFrame(l,schema=sc).printSchema()
spark.createDataFrame(l,schema=sc).show()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- no: long (nullable = true)

+---+----+---+
| id|name| no|
+---+----+---+
|  1| abc| 23|
|  2| xyz| 45|
+---+----+---+



In [16]:
df.show()

+------+--------------------+----------+-----------+--------------------+--------------------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|
+------+--------------------+----------+-----------+--------------------+--------------------+
|178307|            MSI Kart|9319919988|trial_ended|2021-01-02 09:33:...|2021-01-02 09:39:...|
|177863|  Bellelyse Boutique|      null|  suspended|2020-12-31 04:56:...|2021-02-19 16:53:...|
|144524|     Rachid hamzaoui| 639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|
|140294|              myself|7149259700|       free|2020-09-05 17:41:...|2020-10-06 11:15:...|
|165163|  JeeJee’s Boutique |      null|trial_ended|2020-11-20 16:31:...|2020-11-20 16:33:...|
|156444|     Only The Flyest|4049038447|       fre

In [17]:
df.groupBy('state').count().show()

+-----------+-----+
|      state|count|
+-----------+-----+
|  suspended|    3|
|provisioned|    2|
|       free|    3|
|trial_ended|    2|
+-----------+-----+



In [18]:
df.groupBy('state').agg(count('id').alias('count')).show()

+-----------+-----+
|      state|count|
+-----------+-----+
|  suspended|    3|
|provisioned|    2|
|       free|    3|
|trial_ended|    2|
+-----------+-----+



In [19]:
df.groupBy('state').agg(collect_list('created_at')).show(truncate=False)

+-----------+------------------------------------------------------------------------------------+
|state      |collect_list(created_at)                                                            |
+-----------+------------------------------------------------------------------------------------+
|suspended  |[2020-12-31 04:56:24.296919, 2019-03-22 03:57:02.244211, 2020-12-23 06:20:41.788311]|
|provisioned|[2020-10-26 20:10:01.021332, 2019-02-04 16:57:43.781315]                            |
|free       |[2020-09-21 17:18:02.35187, 2020-09-05 17:41:02.896855, 2020-10-27 05:26:41.925243] |
|trial_ended|[2021-01-02 09:33:02.178749, 2020-11-20 16:31:41.993005]                            |
+-----------+------------------------------------------------------------------------------------+



In [20]:
df.groupBy('state').agg(max('created_at')).show(truncate=False)

+-----------+--------------------------+
|state      |max(created_at)           |
+-----------+--------------------------+
|suspended  |2020-12-31 04:56:24.296919|
|provisioned|2020-10-26 20:10:01.021332|
|free       |2020-10-27 05:26:41.925243|
|trial_ended|2021-01-02 09:33:02.178749|
+-----------+--------------------------+



In [21]:
df.select('telephone',coalesce('telephone')).show()

+----------+-------------------+
| telephone|coalesce(telephone)|
+----------+-------------------+
|9319919988|         9319919988|
|      null|               null|
| 639756718|          639756718|
|5049394269|         5049394269|
|6479863690|         6479863690|
|7149259700|         7149259700|
|      null|               null|
|4049038447|         4049038447|
|      null|               null|
|3182596422|         3182596422|
+----------+-------------------+



In [22]:
## coalesce returns first non null column so in this case all values in first col i.e. sum(company_name) was null,
## so it coalesce returned non null value i.e. 0
df.groupBy('state').agg(coalesce(sum('company_name'),lit(0))).show()

+-----------+------------------------------+
|      state|coalesce(sum(company_name), 0)|
+-----------+------------------------------+
|  suspended|                           0.0|
|provisioned|                           0.0|
|       free|                           0.0|
|trial_ended|                           0.0|
+-----------+------------------------------+



In [23]:
df.groupBy('state').agg(approx_count_distinct('id')).show()

21/11/14 08:49:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+-------------------------+
|      state|approx_count_distinct(id)|
+-----------+-------------------------+
|  suspended|                        3|
|provisioned|                        2|
|       free|                        3|
|trial_ended|                        2|
+-----------+-------------------------+



In [24]:
df.groupBy('state').agg(approxCountDistinct('id')).show()

+-----------+-------------------------+
|      state|approx_count_distinct(id)|
+-----------+-------------------------+
|  suspended|                        3|
|provisioned|                        2|
|       free|                        3|
|trial_ended|                        2|
+-----------+-------------------------+



In [25]:
df.select(avg('id')).show()

+--------+
| avg(id)|
+--------+
|138997.7|
+--------+



In [26]:
df.select(collect_list('id')).show(truncate=False)

+------------------------------------------------------------------------------+
|collect_list(id)                                                              |
+------------------------------------------------------------------------------+
|[178307, 177863, 144524, 50742, 176321, 140294, 165163, 156444, 156237, 44082]|
+------------------------------------------------------------------------------+



In [27]:
df.select(countDistinct('id')).show()

+------------------+
|count(DISTINCT id)|
+------------------+
|                10|
+------------------+



In [28]:
df.show()

+------+--------------------+----------+-----------+--------------------+--------------------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|
+------+--------------------+----------+-----------+--------------------+--------------------+
|178307|            MSI Kart|9319919988|trial_ended|2021-01-02 09:33:...|2021-01-02 09:39:...|
|177863|  Bellelyse Boutique|      null|  suspended|2020-12-31 04:56:...|2021-02-19 16:53:...|
|144524|     Rachid hamzaoui| 639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|
|140294|              myself|7149259700|       free|2020-09-05 17:41:...|2020-10-06 11:15:...|
|165163|  JeeJee’s Boutique |      null|trial_ended|2020-11-20 16:31:...|2020-11-20 16:33:...|
|156444|     Only The Flyest|4049038447|       fre

In [29]:
df.createOrReplaceTempView('df')

In [30]:
spark.sql("""select id, company_name, state, 
        row_number() over (partition by state order by id desc) as row_num from df""").show()

+------+--------------------+-----------+-------+
|    id|        company_name|      state|row_num|
+------+--------------------+-----------+-------+
|177863|  Bellelyse Boutique|  suspended|      1|
|176321|   CLICK MY CART LTD|  suspended|      2|
| 50742|Thenaricalicollec...|  suspended|      3|
|156237|          Washer Fan|provisioned|      1|
| 44082|Quick Draw Outdoo...|provisioned|      2|
|156444|     Only The Flyest|       free|      1|
|144524|     Rachid hamzaoui|       free|      2|
|140294|              myself|       free|      3|
|178307|            MSI Kart|trial_ended|      1|
|165163|  JeeJee’s Boutique |trial_ended|      2|
+------+--------------------+-----------+-------+



In [31]:
from pyspark.sql import Window

In [32]:
windowSpec = Window.partitionBy('state').orderBy('id')
df.select('id','company_name','state',row_number().over(windowSpec).alias('row_num')).show()

+------+--------------------+-----------+-------+
|    id|        company_name|      state|row_num|
+------+--------------------+-----------+-------+
| 50742|Thenaricalicollec...|  suspended|      1|
|176321|   CLICK MY CART LTD|  suspended|      2|
|177863|  Bellelyse Boutique|  suspended|      3|
| 44082|Quick Draw Outdoo...|provisioned|      1|
|156237|          Washer Fan|provisioned|      2|
|140294|              myself|       free|      1|
|144524|     Rachid hamzaoui|       free|      2|
|156444|     Only The Flyest|       free|      3|
|165163|  JeeJee’s Boutique |trial_ended|      1|
|178307|            MSI Kart|trial_ended|      2|
+------+--------------------+-----------+-------+



In [33]:
## rank leaves gap in ranking when there are ties
windowSpec = Window.partitionBy('state').orderBy('id')
df.select('id','company_name','state',rank().over(windowSpec).alias('ranking')).show()

+------+--------------------+-----------+-------+
|    id|        company_name|      state|ranking|
+------+--------------------+-----------+-------+
| 50742|Thenaricalicollec...|  suspended|      1|
|176321|   CLICK MY CART LTD|  suspended|      2|
|177863|  Bellelyse Boutique|  suspended|      3|
| 44082|Quick Draw Outdoo...|provisioned|      1|
|156237|          Washer Fan|provisioned|      2|
|140294|              myself|       free|      1|
|144524|     Rachid hamzaoui|       free|      2|
|156444|     Only The Flyest|       free|      3|
|165163|  JeeJee’s Boutique |trial_ended|      1|
|178307|            MSI Kart|trial_ended|      2|
+------+--------------------+-----------+-------+



In [34]:
spark.sql("""select id, company_name, state,
        rank() over(partition by state order by id) as ranking from df""").show()

+------+--------------------+-----------+-------+
|    id|        company_name|      state|ranking|
+------+--------------------+-----------+-------+
| 50742|Thenaricalicollec...|  suspended|      1|
|176321|   CLICK MY CART LTD|  suspended|      2|
|177863|  Bellelyse Boutique|  suspended|      3|
| 44082|Quick Draw Outdoo...|provisioned|      1|
|156237|          Washer Fan|provisioned|      2|
|140294|              myself|       free|      1|
|144524|     Rachid hamzaoui|       free|      2|
|156444|     Only The Flyest|       free|      3|
|165163|  JeeJee’s Boutique |trial_ended|      1|
|178307|            MSI Kart|trial_ended|      2|
+------+--------------------+-----------+-------+



In [35]:
## dense rank doesnt leaves gap in ranking when there are ties
windowSpec = Window.partitionBy('state').orderBy('id')
df.select('id','company_name','state',dense_rank().over(windowSpec).alias('dense_Ranking')).show()

+------+--------------------+-----------+-------------+
|    id|        company_name|      state|dense_Ranking|
+------+--------------------+-----------+-------------+
| 50742|Thenaricalicollec...|  suspended|            1|
|176321|   CLICK MY CART LTD|  suspended|            2|
|177863|  Bellelyse Boutique|  suspended|            3|
| 44082|Quick Draw Outdoo...|provisioned|            1|
|156237|          Washer Fan|provisioned|            2|
|140294|              myself|       free|            1|
|144524|     Rachid hamzaoui|       free|            2|
|156444|     Only The Flyest|       free|            3|
|165163|  JeeJee’s Boutique |trial_ended|            1|
|178307|            MSI Kart|trial_ended|            2|
+------+--------------------+-----------+-------------+



In [36]:
windowSpec = Window.partitionBy('state').orderBy('id')
df.select('id','company_name','state',lag('id',1).over(windowSpec).alias('lag'),lead('id',1).over(windowSpec).alias('lead')).show()

+------+--------------------+-----------+------+------+
|    id|        company_name|      state|   lag|  lead|
+------+--------------------+-----------+------+------+
| 50742|Thenaricalicollec...|  suspended|  null|176321|
|176321|   CLICK MY CART LTD|  suspended| 50742|177863|
|177863|  Bellelyse Boutique|  suspended|176321|  null|
| 44082|Quick Draw Outdoo...|provisioned|  null|156237|
|156237|          Washer Fan|provisioned| 44082|  null|
|140294|              myself|       free|  null|144524|
|144524|     Rachid hamzaoui|       free|140294|156444|
|156444|     Only The Flyest|       free|144524|  null|
|165163|  JeeJee’s Boutique |trial_ended|  null|178307|
|178307|            MSI Kart|trial_ended|165163|  null|
+------+--------------------+-----------+------+------+



In [37]:
windowSpec = Window.partitionBy('state').orderBy('id')
df.withColumn("row",row_number().over(windowSpec)).where(col('row') == 1).select('id','company_name','state').show()

+------+--------------------+-----------+
|    id|        company_name|      state|
+------+--------------------+-----------+
| 50742|Thenaricalicollec...|  suspended|
| 44082|Quick Draw Outdoo...|provisioned|
|140294|              myself|       free|
|165163|  JeeJee’s Boutique |trial_ended|
+------+--------------------+-----------+



In [38]:
df.where(col('state')=='free').show()

+------+---------------+----------+-----+--------------------+--------------------+
|    id|   company_name| telephone|state|          created_at|          updated_at|
+------+---------------+----------+-----+--------------------+--------------------+
|144524|Rachid hamzaoui| 639756718| free|2020-09-21 17:18:...|2020-10-22 11:15:...|
|140294|         myself|7149259700| free|2020-09-05 17:41:...|2020-10-06 11:15:...|
|156444|Only The Flyest|4049038447| free|2020-10-27 05:26:...|2020-11-26 11:15:...|
+------+---------------+----------+-----+--------------------+--------------------+



In [39]:
windowSpecAgg = Window.partitionBy('state')
df.withColumn('row',row_number().over(windowSpec))\
.withColumn('max_id',max(col('id')).over(windowSpecAgg)) \
.select('id','company_name','state','max_id') \
.where(col('row')==1).show()

+------+--------------------+-----------+------+
|    id|        company_name|      state|max_id|
+------+--------------------+-----------+------+
| 50742|Thenaricalicollec...|  suspended|177863|
| 44082|Quick Draw Outdoo...|provisioned|156237|
|140294|              myself|       free|156444|
|165163|  JeeJee’s Boutique |trial_ended|178307|
+------+--------------------+-----------+------+



In [40]:
df.select('created_at',date_format(col('created_at'),'dd-MM-yyyy').alias('created_date'), to_date('created_at').alias('created_date_new')).show(truncate=False)

+--------------------------+------------+----------------+
|created_at                |created_date|created_date_new|
+--------------------------+------------+----------------+
|2021-01-02 09:33:02.178749|02-01-2021  |2021-01-02      |
|2020-12-31 04:56:24.296919|31-12-2020  |2020-12-31      |
|2020-09-21 17:18:02.35187 |21-09-2020  |2020-09-21      |
|2019-03-22 03:57:02.244211|22-03-2019  |2019-03-22      |
|2020-12-23 06:20:41.788311|23-12-2020  |2020-12-23      |
|2020-09-05 17:41:02.896855|05-09-2020  |2020-09-05      |
|2020-11-20 16:31:41.993005|20-11-2020  |2020-11-20      |
|2020-10-27 05:26:41.925243|27-10-2020  |2020-10-27      |
|2020-10-26 20:10:01.021332|26-10-2020  |2020-10-26      |
|2019-02-04 16:57:43.781315|04-02-2019  |2019-02-04      |
+--------------------------+------------+----------------+



In [41]:
## add_months() to the column by adding months
df.select(date_format('created_at','dd-MM-yyyy').alias('created_date'),add_months('created_at',2).alias('updated_date')).show()

+------------+------------+
|created_date|updated_date|
+------------+------------+
|  02-01-2021|  2021-03-02|
|  31-12-2020|  2021-02-28|
|  21-09-2020|  2020-11-21|
|  22-03-2019|  2019-05-22|
|  23-12-2020|  2021-02-23|
|  05-09-2020|  2020-11-05|
|  20-11-2020|  2021-01-20|
|  27-10-2020|  2020-12-27|
|  26-10-2020|  2020-12-26|
|  04-02-2019|  2019-04-04|
+------------+------------+



In [42]:
df.select(months_between('updated_at','created_at')).show()

+--------------------------------------------+
|months_between(updated_at, created_at, true)|
+--------------------------------------------+
|                                         0.0|
|                                  1.62896431|
|                                  1.02412821|
|                                  1.77011387|
|                                  0.03029458|
|                                  1.02361447|
|                                         0.0|
|                                  0.97555742|
|                                         0.0|
|                                 24.17359991|
+--------------------------------------------+



In [43]:
## day_of_week  -> 1 for sunday , 7 for saturday, day of month is same as day in the date, 
df.select(date_format('created_at','dd-MM-yyyy').alias('created_date'), \
         year('created_at').alias('year'), \
         month('created_at').alias('month'),
         dayofweek('created_at').alias('day_of_week'), \
         dayofmonth('created_at').alias('day_of_month'), \
         dayofyear('created_at').alias('day_of_year'), \
         last_day('created_at').alias('last_Day_of_month'), \
         hour('created_at').alias('hour')).show()

+------------+----+-----+-----------+------------+-----------+-----------------+----+
|created_date|year|month|day_of_week|day_of_month|day_of_year|last_Day_of_month|hour|
+------------+----+-----+-----------+------------+-----------+-----------------+----+
|  02-01-2021|2021|    1|          7|           2|          2|       2021-01-31|   9|
|  31-12-2020|2020|   12|          5|          31|        366|       2020-12-31|   4|
|  21-09-2020|2020|    9|          2|          21|        265|       2020-09-30|  17|
|  22-03-2019|2019|    3|          6|          22|         81|       2019-03-31|   3|
|  23-12-2020|2020|   12|          4|          23|        358|       2020-12-31|   6|
|  05-09-2020|2020|    9|          7|           5|        249|       2020-09-30|  17|
|  20-11-2020|2020|   11|          6|          20|        325|       2020-11-30|  16|
|  27-10-2020|2020|   10|          3|          27|        301|       2020-10-31|   5|
|  26-10-2020|2020|   10|          2|          26|    

In [44]:
jsonString="""{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}"""
df_json=spark.createDataFrame([(1, jsonString)],["id","value"])
df_json.show(truncate=False)

+---+--------------------------------------------------------------------------+
|id |value                                                                     |
+---+--------------------------------------------------------------------------+
|1  |{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}|
+---+--------------------------------------------------------------------------+



In [45]:
df_json.printSchema()

root
 |-- id: long (nullable = true)
 |-- value: string (nullable = true)



In [46]:
from pyspark.sql.types import *

In [47]:
df2=df_json.withColumn("value",from_json(df_json.value,MapType(StringType(),StringType())))
df2.printSchema()
df2.show(truncate=False)

root
 |-- id: long (nullable = true)
 |-- value: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+---+---------------------------------------------------------------------------+
|id |value                                                                      |
+---+---------------------------------------------------------------------------+
|1  |{Zipcode -> 704, ZipCodeType -> STANDARD, City -> PARC PARQUE, State -> PR}|
+---+---------------------------------------------------------------------------+



In [48]:
df.show()

+------+--------------------+----------+-----------+--------------------+--------------------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|
+------+--------------------+----------+-----------+--------------------+--------------------+
|178307|            MSI Kart|9319919988|trial_ended|2021-01-02 09:33:...|2021-01-02 09:39:...|
|177863|  Bellelyse Boutique|      null|  suspended|2020-12-31 04:56:...|2021-02-19 16:53:...|
|144524|     Rachid hamzaoui| 639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|
|140294|              myself|7149259700|       free|2020-09-05 17:41:...|2020-10-06 11:15:...|
|165163|  JeeJee’s Boutique |      null|trial_ended|2020-11-20 16:31:...|2020-11-20 16:33:...|
|156444|     Only The Flyest|4049038447|       fre

In [49]:
df.withColumn('new_state', when(col('state') == 'free', "terminated")
                        .when(col('state') == 'suspended', "waiting")
                        .otherwise('same as before')) \
                        .select('id','company_name','state','new_state').show()

+------+--------------------+-----------+--------------+
|    id|        company_name|      state|     new_state|
+------+--------------------+-----------+--------------+
|178307|            MSI Kart|trial_ended|same as before|
|177863|  Bellelyse Boutique|  suspended|       waiting|
|144524|     Rachid hamzaoui|       free|    terminated|
| 50742|Thenaricalicollec...|  suspended|       waiting|
|176321|   CLICK MY CART LTD|  suspended|       waiting|
|140294|              myself|       free|    terminated|
|165163|  JeeJee’s Boutique |trial_ended|same as before|
|156444|     Only The Flyest|       free|    terminated|
|156237|          Washer Fan|provisioned|same as before|
| 44082|Quick Draw Outdoo...|provisioned|same as before|
+------+--------------------+-----------+--------------+



In [50]:
spark.sql("""select id, company_name, state,
            case when state = 'free' then 'terminated'
                when state = 'suspended' then 'waiting'
                else 'same as before'
            end as new_state from df""").show()

+------+--------------------+-----------+--------------+
|    id|        company_name|      state|     new_state|
+------+--------------------+-----------+--------------+
|178307|            MSI Kart|trial_ended|same as before|
|177863|  Bellelyse Boutique|  suspended|       waiting|
|144524|     Rachid hamzaoui|       free|    terminated|
| 50742|Thenaricalicollec...|  suspended|       waiting|
|176321|   CLICK MY CART LTD|  suspended|       waiting|
|140294|              myself|       free|    terminated|
|165163|  JeeJee’s Boutique |trial_ended|same as before|
|156444|     Only The Flyest|       free|    terminated|
|156237|          Washer Fan|provisioned|same as before|
| 44082|Quick Draw Outdoo...|provisioned|same as before|
+------+--------------------+-----------+--------------+



In [51]:
df.withColumn("new_id",lit(5)).show()

+------+--------------------+----------+-----------+--------------------+--------------------+------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|new_id|
+------+--------------------+----------+-----------+--------------------+--------------------+------+
|178307|            MSI Kart|9319919988|trial_ended|2021-01-02 09:33:...|2021-01-02 09:39:...|     5|
|177863|  Bellelyse Boutique|      null|  suspended|2020-12-31 04:56:...|2021-02-19 16:53:...|     5|
|144524|     Rachid hamzaoui| 639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|     5|
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|     5|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|     5|
|140294|              myself|7149259700|       free|2020-09-05 17:41:...|2020-10-06 11:15:...|     5|
|165163|  JeeJee’s Boutique |      null|trial_ended|2020-11-20 16:31:...|2020-11-2

In [52]:
## split the column into an array type and then explode the array elements into separate rows
df.select('company_name',explode(split('company_name',' ')).alias('new_company_name')).show(truncate=False)

+---------------------+---------------------+
|company_name         |new_company_name     |
+---------------------+---------------------+
|MSI Kart             |MSI                  |
|MSI Kart             |Kart                 |
|Bellelyse Boutique   |Bellelyse            |
|Bellelyse Boutique   |Boutique             |
|Rachid hamzaoui      |Rachid               |
|Rachid hamzaoui      |hamzaoui             |
|Thenaricalicollection|Thenaricalicollection|
|CLICK MY CART LTD    |CLICK                |
|CLICK MY CART LTD    |MY                   |
|CLICK MY CART LTD    |CART                 |
|CLICK MY CART LTD    |LTD                  |
|myself               |myself               |
|JeeJee’s Boutique    |JeeJee’s             |
|JeeJee’s Boutique    |Boutique             |
|JeeJee’s Boutique    |                     |
|Only The Flyest      |Only                 |
|Only The Flyest      |The                  |
|Only The Flyest      |Flyest               |
|Washer Fan           |Washer     

In [53]:
## convert array column to string
df.select('company_name',concat_ws(',',split('company_name',' ')).alias('new_company_name')).show(truncate=False)

+---------------------------+---------------------------+
|company_name               |new_company_name           |
+---------------------------+---------------------------+
|MSI Kart                   |MSI,Kart                   |
|Bellelyse Boutique         |Bellelyse,Boutique         |
|Rachid hamzaoui            |Rachid,hamzaoui            |
|Thenaricalicollection      |Thenaricalicollection      |
|CLICK MY CART LTD          |CLICK,MY,CART,LTD          |
|myself                     |myself                     |
|JeeJee’s Boutique          |JeeJee’s,Boutique,         |
|Only The Flyest            |Only,The,Flyest            |
|Washer Fan                 |Washer,Fan                 |
|Quick Draw Outdoor Gear llc|Quick,Draw,Outdoor,Gear,llc|
+---------------------------+---------------------------+



In [54]:
## using substring by giving column name , starting position, and length of substring
df.select('state',substring('state',1,2).alias('new_state')).show()

+-----------+---------+
|      state|new_state|
+-----------+---------+
|trial_ended|       tr|
|  suspended|       su|
|       free|       fr|
|  suspended|       su|
|  suspended|       su|
|       free|       fr|
|trial_ended|       tr|
|       free|       fr|
|provisioned|       pr|
|provisioned|       pr|
+-----------+---------+



In [55]:
## translate every character present in the column with another character given as parameter
df.select('state',translate('state','abcdefghij','123456789').alias('translator')).show(truncate=False)

+-----------+-----------+
|state      |translator |
+-----------+-----------+
|trial_ended|tr91l_5n454|
|suspended  |susp5n454  |
|free       |6r55       |
|suspended  |susp5n454  |
|suspended  |susp5n454  |
|free       |6r55       |
|trial_ended|tr91l_5n454|
|free       |6r55       |
|provisioned|prov9s9on54|
|provisioned|prov9s9on54|
+-----------+-----------+



In [56]:
## replace column value with the 3rd parameter by matching column value with the second parameter and ignore if 
## doesnt matches
df.select('state',regexp_replace('state','_',' ').alias('new_State')).show()

+-----------+-----------+
|      state|  new_State|
+-----------+-----------+
|trial_ended|trial ended|
|  suspended|  suspended|
|       free|       free|
|  suspended|  suspended|
|  suspended|  suspended|
|       free|       free|
|trial_ended|trial ended|
|       free|       free|
|provisioned|provisioned|
|provisioned|provisioned|
+-----------+-----------+



In [57]:
## replacing column value of state with another value starting from the index mentioned in the 3rd parameter
df.select('state',overlay('state',lit('6'),1).alias('overlayed')).show()

+-----------+-----------+
|      state|  overlayed|
+-----------+-----------+
|trial_ended|6rial_ended|
|  suspended|  6uspended|
|       free|       6ree|
|  suspended|  6uspended|
|  suspended|  6uspended|
|       free|       6ree|
|trial_ended|6rial_ended|
|       free|       6ree|
|provisioned|6rovisioned|
|provisioned|6rovisioned|
+-----------+-----------+



In [58]:
df.withColumn('new_company_name',split('company_name',' ')) \
    .select('company_name',col('new_company_name').getItem(0).alias('first_name') \
            ,col('new_company_name').getItem(1).alias('last_name') \
           ,array_contains('new_company_name','MSI').alias('array_contains')).show()

+--------------------+--------------------+---------+--------------+
|        company_name|          first_name|last_name|array_contains|
+--------------------+--------------------+---------+--------------+
|            MSI Kart|                 MSI|     Kart|          true|
|  Bellelyse Boutique|           Bellelyse| Boutique|         false|
|     Rachid hamzaoui|              Rachid| hamzaoui|         false|
|Thenaricalicollec...|Thenaricalicollec...|     null|         false|
|   CLICK MY CART LTD|               CLICK|       MY|         false|
|              myself|              myself|     null|         false|
|  JeeJee’s Boutique |            JeeJee’s| Boutique|         false|
|     Only The Flyest|                Only|      The|         false|
|          Washer Fan|              Washer|      Fan|         false|
|Quick Draw Outdoo...|               Quick|     Draw|         false|
+--------------------+--------------------+---------+--------------+



In [59]:
## all col values in list
df.select('state').rdd.map(lambda x:x[0]).collect()

['trial_ended',
 'suspended',
 'free',
 'suspended',
 'suspended',
 'free',
 'trial_ended',
 'free',
 'provisioned',
 'provisioned']

In [60]:
df.distinct().count()

10

In [61]:
df.select(countDistinct('id')).show()

+------------------+
|count(DISTINCT id)|
+------------------+
|                10|
+------------------+



In [62]:
## coalesce returns the first non null column
df.select('telephone',coalesce(col('telephone'),lit(0))).show()

+----------+----------------------+
| telephone|coalesce(telephone, 0)|
+----------+----------------------+
|9319919988|            9319919988|
|      null|                     0|
| 639756718|             639756718|
|5049394269|            5049394269|
|6479863690|            6479863690|
|7149259700|            7149259700|
|      null|                     0|
|4049038447|            4049038447|
|      null|                     0|
|3182596422|            3182596422|
+----------+----------------------+



In [63]:
## replaces all null values from telephone column to -1 using when otherwise stmt
df.withColumn('telephone',when(col('telephone').isNull(), -1).otherwise(col('telephone'))).show()

+------+--------------------+----------+-----------+--------------------+--------------------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|
+------+--------------------+----------+-----------+--------------------+--------------------+
|178307|            MSI Kart|9319919988|trial_ended|2021-01-02 09:33:...|2021-01-02 09:39:...|
|177863|  Bellelyse Boutique|        -1|  suspended|2020-12-31 04:56:...|2021-02-19 16:53:...|
|144524|     Rachid hamzaoui| 639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|
|140294|              myself|7149259700|       free|2020-09-05 17:41:...|2020-10-06 11:15:...|
|165163|  JeeJee’s Boutique |        -1|trial_ended|2020-11-20 16:31:...|2020-11-20 16:33:...|
|156444|     Only The Flyest|4049038447|       fre

In [64]:
df.select([when(col(c).isNull(), 'NA').otherwise(col(c)).alias(c) for c in df.columns]).show()

+------+--------------------+----------+-----------+--------------------+--------------------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|
+------+--------------------+----------+-----------+--------------------+--------------------+
|178307|            MSI Kart|9319919988|trial_ended|2021-01-02 09:33:...|2021-01-02 09:39:...|
|177863|  Bellelyse Boutique|        NA|  suspended|2020-12-31 04:56:...|2021-02-19 16:53:...|
|144524|     Rachid hamzaoui| 639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|
|140294|              myself|7149259700|       free|2020-09-05 17:41:...|2020-10-06 11:15:...|
|165163|  JeeJee’s Boutique |        NA|trial_ended|2020-11-20 16:31:...|2020-11-20 16:33:...|
|156444|     Only The Flyest|4049038447|       fre

In [65]:
df.fillna(-1).show()

+------+--------------------+----------+-----------+--------------------+--------------------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|
+------+--------------------+----------+-----------+--------------------+--------------------+
|178307|            MSI Kart|9319919988|trial_ended|2021-01-02 09:33:...|2021-01-02 09:39:...|
|177863|  Bellelyse Boutique|        -1|  suspended|2020-12-31 04:56:...|2021-02-19 16:53:...|
|144524|     Rachid hamzaoui| 639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|
|140294|              myself|7149259700|       free|2020-09-05 17:41:...|2020-10-06 11:15:...|
|165163|  JeeJee’s Boutique |        -1|trial_ended|2020-11-20 16:31:...|2020-11-20 16:33:...|
|156444|     Only The Flyest|4049038447|       fre

In [66]:
df.na.fill(-1).show()

+------+--------------------+----------+-----------+--------------------+--------------------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|
+------+--------------------+----------+-----------+--------------------+--------------------+
|178307|            MSI Kart|9319919988|trial_ended|2021-01-02 09:33:...|2021-01-02 09:39:...|
|177863|  Bellelyse Boutique|        -1|  suspended|2020-12-31 04:56:...|2021-02-19 16:53:...|
|144524|     Rachid hamzaoui| 639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|
|140294|              myself|7149259700|       free|2020-09-05 17:41:...|2020-10-06 11:15:...|
|165163|  JeeJee’s Boutique |        -1|trial_ended|2020-11-20 16:31:...|2020-11-20 16:33:...|
|156444|     Only The Flyest|4049038447|       fre

In [67]:
df.na.fill(0,['telephone']).show()

+------+--------------------+----------+-----------+--------------------+--------------------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|
+------+--------------------+----------+-----------+--------------------+--------------------+
|178307|            MSI Kart|9319919988|trial_ended|2021-01-02 09:33:...|2021-01-02 09:39:...|
|177863|  Bellelyse Boutique|         0|  suspended|2020-12-31 04:56:...|2021-02-19 16:53:...|
|144524|     Rachid hamzaoui| 639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|
|140294|              myself|7149259700|       free|2020-09-05 17:41:...|2020-10-06 11:15:...|
|165163|  JeeJee’s Boutique |         0|trial_ended|2020-11-20 16:31:...|2020-11-20 16:33:...|
|156444|     Only The Flyest|4049038447|       fre

In [68]:
df.rdd.map(lambda x : (x[0],x[1]+ '--' + x[3])).collect()

[(178307, 'MSI Kart--trial_ended'),
 (177863, 'Bellelyse Boutique--suspended'),
 (144524, 'Rachid hamzaoui--free'),
 (50742, 'Thenaricalicollection--suspended'),
 (176321, 'CLICK MY CART LTD--suspended'),
 (140294, 'myself--free'),
 (165163, 'JeeJee’s Boutique --trial_ended'),
 (156444, 'Only The Flyest--free'),
 (156237, 'Washer Fan--provisioned'),
 (44082, 'Quick Draw Outdoor Gear llc--provisioned')]

In [82]:
df.select('id',concat_ws(' -- ','company_name','state').alias('concat')).show(truncate=False)

+------+------------------------------------------+
|id    |concat                                    |
+------+------------------------------------------+
|178307|MSI Kart -- trial_ended                   |
|177863|Bellelyse Boutique -- suspended           |
|144524|Rachid hamzaoui -- free                   |
|50742 |Thenaricalicollection -- suspended        |
|176321|CLICK MY CART LTD -- suspended            |
|140294|myself -- free                            |
|165163|JeeJee’s Boutique  -- trial_ended         |
|156444|Only The Flyest -- free                   |
|156237|Washer Fan -- provisioned                 |
|44082 |Quick Draw Outdoor Gear llc -- provisioned|
+------+------------------------------------------+



In [69]:
df.rdd.map(lambda x : (x[0],x[1]+ ' -- ' + x['state'])).toDF(['id','concat']).show(truncate=False)

+------+------------------------------------------+
|id    |concat                                    |
+------+------------------------------------------+
|178307|MSI Kart -- trial_ended                   |
|177863|Bellelyse Boutique -- suspended           |
|144524|Rachid hamzaoui -- free                   |
|50742 |Thenaricalicollection -- suspended        |
|176321|CLICK MY CART LTD -- suspended            |
|140294|myself -- free                            |
|165163|JeeJee’s Boutique  -- trial_ended         |
|156444|Only The Flyest -- free                   |
|156237|Washer Fan -- provisioned                 |
|44082 |Quick Draw Outdoor Gear llc -- provisioned|
+------+------------------------------------------+



In [70]:
df.foreach(lambda x : (x[0],x[1]+'--'+x[3]))

In [71]:
## to enable apache arrow with spark
## spark uses arrow which is an in-memory columnar format to transfer data between python and jvm
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled','true')

In [72]:
pandasDF = df.toPandas()

In [73]:
pandasDF.head()

,id,company_name,telephone,state,created_at,updated_at
0,178307,MSI Kart,9.319920e+09,trial_ended,2021-01-02 09:33:02.178749,2021-01-02 09:39:08.302584
1,177863,Bellelyse Boutique,NaN,suspended,2020-12-31 04:56:24.296919,2021-02-19 16:53:22.503263
2,144524,Rachid hamzaoui,6.397567e+08,free,2020-09-21 17:18:02.351870,2020-10-22 11:15:07.122165
3,50742,Thenaricalicollection,5.049394e+09,suspended,2019-03-22 03:57:02.244211,2019-05-15 00:54:55.439291
4,176321,CLICK MY CART LTD,6.479864e+09,suspended,2020-12-23 06:20:41.788311,2020-12-24 04:53:02.186645


In [74]:
pandasDF.dtypes

id                       int32
company_name            object
telephone              float64
state                   object
created_at      datetime64[ns]
updated_at      datetime64[ns]
dtype: object

In [86]:
## to make udf first create python function 
def convertCase(string):
    res = ''
    for s in string.split('_'):
        res += s[0:1].upper() + s[1:] + '_'
    return res[:-1]

In [87]:
convertCase('trail_ended')

'Trail_Ended'

In [89]:
# now register python function to pyspark udf
converUDF = udf(convertCase,StringType())

In [90]:
df.select('state',converUDF('state').alias('convertUDFcase')).show()

+-----------+--------------+
|      state|convertUDFcase|
+-----------+--------------+
|trial_ended|   Trial_Ended|
|  suspended|     Suspended|
|       free|          Free|
|  suspended|     Suspended|
|  suspended|     Suspended|
|       free|          Free|
|trial_ended|   Trial_Ended|
|       free|          Free|
|provisioned|   Provisioned|
|provisioned|   Provisioned|
+-----------+--------------+



In [92]:
def upperCase(string):
    return string.upper()

upperUDF = udf(upperCase,StringType())

In [93]:
df.select('state',upperUDF('state').alias('upper_state')).show()

+-----------+-----------+
|      state|upper_state|
+-----------+-----------+
|trial_ended|TRIAL_ENDED|
|  suspended|  SUSPENDED|
|       free|       FREE|
|  suspended|  SUSPENDED|
|  suspended|  SUSPENDED|
|       free|       FREE|
|trial_ended|TRIAL_ENDED|
|       free|       FREE|
|provisioned|PROVISIONED|
|provisioned|PROVISIONED|
+-----------+-----------+



In [94]:
## another method for udf...in pyspark sql is by registering it
spark.udf.register('converUDF_sql', convertCase, StringType())

<function __main__.convertCase(string)>

In [95]:
spark.sql("""select state, converUDF_sql(state) as ConverUDFcase from df""").show()

+-----------+-------------+
|      state|ConverUDFcase|
+-----------+-------------+
|trial_ended|  Trial_Ended|
|  suspended|    Suspended|
|       free|         Free|
|  suspended|    Suspended|
|  suspended|    Suspended|
|       free|         Free|
|trial_ended|  Trial_Ended|
|       free|         Free|
|provisioned|  Provisioned|
|provisioned|  Provisioned|
+-----------+-------------+



In [96]:
spark.udf.register('upperUDF_sql',upperCase, StringType())

<function __main__.upperCase(string)>

In [97]:
spark.sql("""select state, upperUDF_sql(state) as upperUDF from df""").show()

+-----------+-----------+
|      state|   upperUDF|
+-----------+-----------+
|trial_ended|TRIAL_ENDED|
|  suspended|  SUSPENDED|
|       free|       FREE|
|  suspended|  SUSPENDED|
|  suspended|  SUSPENDED|
|       free|       FREE|
|trial_ended|TRIAL_ENDED|
|       free|       FREE|
|provisioned|PROVISIONED|
|provisioned|PROVISIONED|
+-----------+-----------+

